# Google Sheets API

Project involving the Google Sheets API to save data from a Sheets file that is integrated with Forms.

In [1]:
# importing necessary libraries and packages
from __future__ import print_function

import configparser
import os
import shutil

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pandas as pd

In [ ]:
# Variables
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.txt')

# Get the values from the config file
SAMPLE_SPREADSHEET_ID = config.get('SHEET_CONFIG', "SAMPLE_SPREADSHEET_ID")
SAMPLE_RANGE_NAME = config.get('SHEET_CONFIG', 'SAMPLE_RANGE_NAME')


# Read the remote directory path from the text file
with open('remote_dir.txt', 'r') as file:
    remote_dir= file.read().strip()


In [ ]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
    
try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    
    else:
        headers = values[0]
        data = values[1:]
        df = pd.DataFrame(data, columns=headers)
        
        # Save the DataFrame to a local file
        local_file_path = 'urgo_aesthetic_academy.xlsx'
        df.to_excel(local_file_path, index=False)

        # Move the file to the remote directory
        remote_file_path = os.path.join(remote_dir, local_file_path)
        shutil.move(local_file_path, remote_file_path)

        print(f'Saved DataFrame as on the remote server')
except HttpError as err:
    print(err)



Saved DataFrame as on the remote server
